<a href="https://colab.research.google.com/github/sushirito/Methylmercury/blob/main/Diffusion_Boltzmann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This code was inspired by: https://github.com/lbm-principles-practice/code/blob/master/chapter9/shanchen.cpp



In [5]:
#Fixed in accordance w/paper

import numpy as np
import plotly.graph_objects as go
import os
import matplotlib.pyplot as plt

# Lattice parameters
nx, ny, nz = 20, 20, 20  # Domain size matching Lx, Ly, Lz
nsteps = 100  # Number of time steps matching the given t = 4000
noutput = 20  # Output frequency

# D3Q7 model parameters
npop = 7  # Number of populations
cx = np.array([0, 1, -1, 0, 0, 0, 0])
cy = np.array([0, 0, 0, 1, -1, 0, 0])
cz = np.array([0, 0, 0, 0, 0, 1, -1])
weight = np.array([1 / 4, 1 / 8, 1 / 8, 1 / 8, 1 / 8, 1 / 8, 1 / 8])  # Weights for D3Q7

# Diffusion parameters
D0 = 0.5  # Molecular diffusion coefficient [l.u.]^2 / [t.u.]
Lambda_eo = 1 / 32  # As given in the paper
cs_sq = 1 / 3  # Square of the lattice speed of sound

lambda_o = -2 * cs_sq / (2 * D0 + cs_sq)
Lambda_o = -(1 / 2 + 1 / lambda_o)
Lambda_e = Lambda_eo / Lambda_o
lambda_e = -2 / (2 * Lambda_e + 1)

# Initialize arrays
rho = np.zeros(nx * ny * nz)  # Density (concentration)
f1 = np.zeros((nx * ny * nz, npop))  # Distribution functions
f2 = np.zeros((nx * ny * nz, npop))  # Temporary array for streaming

def create_elementary_pattern(Le):
    pattern = np.zeros((Le, Le, Le), dtype=bool)
    radius = Le // 2
    corners = [(0, 0, 0), (0, 0, Le), (0, Le, 0), (Le, 0, 0),
               (0, Le, Le), (Le, 0, Le), (Le, Le, 0), (Le, Le, Le)]

    for corner in corners:
        cx, cy, cz = corner
        x, y, z = np.ogrid[0:Le, 0:Le, 0:Le]
        dist_from_corner = (x - cx) ** 2 + (y - cy) ** 2 + (z - cz) ** 2
        pattern |= (dist_from_corner <= radius ** 2)

    return pattern

def create_porous_medium(N, Le):
    elementary_pattern = create_elementary_pattern(Le)
    Lm = int(N * Le)
    medium = np.tile(elementary_pattern, (int(np.ceil(N)), int(np.ceil(N)), int(np.ceil(N))))
    return medium[:Lm, :Lm, :Lm]

def calculate_porosity(medium):
    return 1 - np.mean(medium)

# Parameters
Le = 10
N = 2  # Adjusted for larger domain size

# Create porous medium
solid = create_porous_medium(N, Le)

# Calculate porosity
porosity = calculate_porosity(solid)
print(f"Porosity: {porosity:.3f}")

# Equilibrium functions
def equilibrium(rho_local):
    feq = np.zeros(npop)
    feq[0] = weight[0] * rho_local
    for q in range(1, npop):
        feq[q] = weight[q] * rho_local
    return feq

def compute_moments():
    global rho
    rho[:] = np.sum(f1, axis=1)

def collision_and_streaming():
    global f1, f2
    for z in range(nz):
        for y in range(ny):
            for x in range(nx):
                k = z * nx * ny + y * nx + x
                if solid[z, y, x] == 0:  # Fluid node
                    feq = equilibrium(rho[k])
                    for i in range(npop):
                        f_plus = 0.5 * (f1[k, i] + f1[k, npop - 1 - i])
                        f_minus = 0.5 * (f1[k, i] - f1[k, npop - 1 - i])
                        n_plus = f_plus - feq[i]
                        n_minus = f_minus

                        x2 = (x + cx[i] + nx) % nx
                        y2 = (y + cy[i] + ny) % ny
                        z2 = (z + cz[i] + nz) % nz
                        k2 = z2 * nx * ny + y2 * nx + x2

                        f2[k2, i] = f1[k, i] + lambda_e * n_plus + lambda_o * n_minus

    f1, f2 = f2, f1

# Initialization
def initialize_diffusion():
    global f1, f2, rho

    # Set initial concentration at the top face z = z0
    z0 = 10
    C0 = 1.0
    rho[:] = 0  # Initialize with zero concentration
    rho[z0 * nx * ny:(z0 + 1) * nx * ny] = C0  # Set initial concentration at z = z0

    f1.fill(0)
    f2.fill(0)
    for k in range(nx * ny * nz):
        if not solid.flatten()[k]:
            feq = equilibrium(rho[k])
            f1[k] = feq
            f2[k] = feq

# Apply boundary conditions
def apply_boundary_conditions(f1, rho):
    apply_x_boundary_conditions(f1)  # Periodic
    apply_y_boundary_conditions(f1)  # Periodic
    apply_z_boundary_conditions(f1, rho)  # Dirichlet

# X-direction boundaries (Periodic)
def apply_x_boundary_conditions(f1):
    for z in range(nz):
        for y in range(ny):
            for i in range(npop):
                k_low_x = z * nx * ny + y * nx + 0
                k_high_x = z * nx * ny + y * nx + (nx - 1)

                # Periodic Boundary Condition in X-direction
                f1[k_low_x, i] = f1[k_high_x, i]
                f1[k_high_x, i] = f1[k_low_x, i]

# Y-direction boundaries (Periodic)
def apply_y_boundary_conditions(f1):
    for z in range(nz):
        for x in range(nx):
            for i in range(npop):
                k_low_y = z * nx * ny + 0 * nx + x
                k_high_y = z * nx * ny + (ny - 1) * nx + x

                # Periodic Boundary Condition in Y-direction
                f1[k_low_y, i] = f1[k_high_y, i]
                f1[k_high_y, i] = f1[k_low_y, i]

# Z-direction boundaries (Dirichlet)
def apply_z_boundary_conditions(f1, rho):
    for y in range(ny):
        for x in range(nx):
            for i in range(npop):
                k_top = 0 * nx * ny + y * nx + x
                k_bottom = (nz - 1) * nx * ny + y * nx + x

                # Top Face (Dirichlet Boundary Condition, initial concentration)
                f1[k_top, i] = equilibrium(0.0)[i]  # Set to zero concentration

                # Bottom Face (Dirichlet Boundary Condition, zero concentration)
                f1[k_bottom, i] = equilibrium(0.0)[i]


# Assuming output_dir is defined as the directory where you want to save the plots
output_dir = 'output_plots'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def save_html(fig, filename):
    filepath = os.path.join(output_dir, filename)
    fig.write_html(filepath)
    print(f"Saved: {filepath}")

def save_png(fig, filename):
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath)
    plt.close(fig)
    print(f"Saved: {filepath}")

def concentration_plot_3d(rho, step):
    # Creating meshgrid for 3D plot
    z, y, x = np.meshgrid(np.arange(nz), np.arange(ny), np.arange(nx))
    scalar_field = rho.reshape((nz, ny, nx)).flatten()

    # Print density information for debugging
    print(f"Min concentration: {np.min(scalar_field)}")
    print(f"Max concentration: {np.max(scalar_field)}")
    print(f"Mean concentration: {np.mean(scalar_field)}")

    # Create a 3D scatter plot
    fig = go.Figure(data=go.Scatter3d(
        x=x.flatten(),
        y=y.flatten(),
        z=z.flatten(),
        mode='markers',
        marker=dict(
            size=3,
            color=scalar_field,
            colorscale='Viridis',
            opacity=0.5,
            colorbar=dict(title='Concentration')
        )
    ))

    # Update layout for better visualization
    fig.update_layout(
        title=f'Concentration Distribution at Step {step}',
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )

    # Show the figure
    fig.show()

    # Save the plot as HTML
    save_html(fig, f'concentration_plot_3d_{step:05d}.html')

def visualize_boundary_nodes(solid):
    fig = go.Figure(data=go.Volume(
        x=np.arange(nx).repeat(ny * nz),
        y=np.tile(np.arange(ny).repeat(nz), nx),
        z=np.tile(np.arange(nz), nx * ny),
        value=solid.flatten(),
        opacity=0.8,  # Change this value to make boundaries more/less visible
        isomin=0.5,
        isomax=1,
        surface_count=3,
        colorscale='Blues'
    ))

    fig.update_layout(
        title='Boundary Nodes Visualization',
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )

    fig.show()
    save_html(fig, 'boundary_nodes_visualization.html')

# Run the main loop and generate plots at each output step
def main_loop():
    initial_mass = np.sum(rho)
    mass_changes = []
    steps = []

    for step in range(nsteps):
        compute_moments()
        collision_and_streaming()
        apply_boundary_conditions(f1, rho)

        if step % noutput == 0:
            current_mass = np.sum(rho)
            mass_change = (current_mass - initial_mass) / initial_mass * 100
            mass_changes.append(mass_change)
            steps.append(step)
            print(f"Step {step}: Average concentration = {np.mean(rho[solid.flatten() == 0])}")
            print(f"Mass change: {mass_change:.6%}")
            print(f"Min concentration: {np.min(rho)}")
            print(f"Max concentration: {np.max(rho)}")
            print(f"Total mass: {current_mass}")
            concentration_plot_3d(rho, step)

    final_mass = np.sum(rho)
    total_mass_change = (final_mass - initial_mass) / initial_mass
    print(f"Total mass change: {total_mass_change:.6%}")

# Visualize the boundary nodes once before running the simulation
visualize_boundary_nodes(solid)

# Initialize and run simulation
initialize_diffusion()
main_loop()

print("Simulation complete.")

Porosity: 0.488


Saved: output_plots/boundary_nodes_visualization.html
Step 0: Average concentration = 0.02151639344262295
Mass change: -7900.000000%
Min concentration: 0.0
Max concentration: 1.0
Total mass: 84.0
Min concentration: 0.0
Max concentration: 1.0
Mean concentration: 0.0105


Saved: output_plots/concentration_plot_3d_00000.html
Step 20: Average concentration = 0.03107578988520392
Mass change: -6482.892068%
Min concentration: -0.19126710874976122
Max concentration: 0.8870644892314794
Total mass: 140.68431726158164
Min concentration: -0.19126710874976122
Max concentration: 0.8870644892314794
Mean concentration: 0.017585539657697706


Saved: output_plots/concentration_plot_3d_00020.html
Step 40: Average concentration = 0.037991688007550074
Mass change: -5779.110227%
Min concentration: -0.16448713271541404
Max concentration: 0.8840565596181387
Total mass: 168.8355909124926
Min concentration: -0.16448713271541404
Max concentration: 0.8840565596181387
Mean concentration: 0.021104448864061574


Saved: output_plots/concentration_plot_3d_00040.html
Step 60: Average concentration = 0.03946220379881939
Mass change: -5619.496529%
Min concentration: -0.16669545605632968
Max concentration: 0.884298987270341
Total mass: 175.22013883253308
Min concentration: -0.16669545605632968
Max concentration: 0.884298987270341
Mean concentration: 0.021902517354066633


Saved: output_plots/concentration_plot_3d_00060.html
Step 80: Average concentration = 0.03835710446023754
Mass change: -5722.702850%
Min concentration: -0.16629673158455438
Max concentration: 0.8842445275400295
Total mass: 171.09188598634185
Min concentration: -0.16629673158455438
Max concentration: 0.8842445275400295
Mean concentration: 0.02138648574829273


Saved: output_plots/concentration_plot_3d_00080.html
Total mass change: -57.316033%
Simulation complete.
